In [1]:
import requests
import sys
import os
from multiprocessing import Pool
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
base_url = 'http://notelections.online'
entry_url = 'http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

In [3]:
additional_headers = ['Регион', 'ТИК', 'УИК']

In [4]:
doc = bs(requests.get(entry_url).text)

In [5]:
data_table_headers = additional_headers + list(map(lambda x: x.text, doc.select('table.table-bordered.table-striped.table-sm tr td:nth-of-type(2):not([colspan])')))

In [6]:
for header in data_table_headers:
    print(header)

Регион
ТИК
УИК
Число избирателей, включенных в список избирателей 
Число избирательных бюллетеней, полученных участковой избирательной комиссией
Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно
Число избирательных бюллетеней, выданных в помещении для голосования в день голосования
Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования
Число погашенных избирательных бюллетеней
Число избирательных бюллетеней в переносных ящиках для голосования
Число бюллетеней в стационарных ящиках для голосования
Число недействительных избирательных бюллетеней
Число действительных избирательных бюллетеней
Число утраченных избирательных бюллетеней
Число избирательных бюллетеней, не учтенных при получении 
Бабурин Сергей Николаевич
Грудинин Павел Николаевич
Жириновский Владимир Вольфович
Путин Владимир Владимирович
Собчак Ксения Анатольевна
Сурайкин Максим Александрович
Титов Борис Юрьевич
Явлинский Григорий Алексеевич


In [7]:
region_links = doc.select('nobr a')

In [8]:
for index, region in enumerate(list(map(lambda x: x.text, region_links))):
    print(f'{index + 1}. {region}')

1. Республика Адыгея (Адыгея)
2. Республика Алтай
3. Республика Башкортостан
4. Республика Бурятия
5. Республика Дагестан
6. Республика Ингушетия
7. Кабардино-Балкарская Республика
8. Республика Калмыкия
9. Карачаево-Черкесская Республика
10. Республика Карелия
11. Республика Коми
12. Республика Крым
13. Республика Марий Эл
14. Республика Мордовия
15. Республика Саха (Якутия)
16. Республика Северная Осетия - Алания
17. Республика Татарстан (Татарстан)
18. Республика Тыва
19. Удмуртская Республика
20. Республика Хакасия
21. Чеченская Республика
22. Чувашская Республика - Чувашия
23. Алтайский край
24. Забайкальский край
25. Камчатский край
26. Краснодарский край
27. Красноярский край
28. Пермский край
29. Приморский край
30. Ставропольский край
31. Хабаровский край
32. Амурская область
33. Архангельская область
34. Астраханская область
35. Белгородская область
36. Брянская область
37. Владимирская область
38. Волгоградская область
39. Вологодская область
40. Воронежская область
41. Иван

In [9]:
result_table = []

In [10]:
def parse_territory_results(territory_doc, region_name, territory_name = 'None'):
    territory_result_trs = territory_doc.select('table#fix-columns-table tr')
    
    territory_precincts_count = len(territory_doc.select('table#fix-columns-table th')) - 3
    
    rows = [[0 for y in range(len(data_table_headers))] for x in range(territory_precincts_count)]
    
    for i, tr in enumerate(territory_result_trs):
        if i == 0:
            ths = tr.select('th')[3:]

            for j, th in enumerate(ths):
                rows[j][0] = region_name
                rows[j][1] = territory_name
                rows[j][2] = th.text.replace('УИК №', '')
            continue

        tds = tr.select('td')[3:]
        
        for j, td in enumerate(tds):
            rows[j][i + 2] = int(td.nobr.text)

    return rows

In [11]:
def process_region_link(region_link):
    region_path = region_link['href']
    region_name = region_link.text

    region_doc = bs(requests.get(f'{base_url}{region_path}').text)

    territory_links = region_doc.select('nobr a')

    if not territory_links:
        return parse_territory_results(region_doc, region_name)

    region_rows = []
    
    for territory_link in territory_links:
        territory_path = territory_link['href']
        territory_name = territory_link.text

        territory_doc = bs(requests.get(f'{base_url}{territory_path}').text)

        territory_rows = parse_territory_results(territory_doc, region_name, territory_name)
        region_rows.extend(territory_rows)
    
    return region_rows

In [12]:
recursion_limit = sys.getrecursionlimit()

# Change recursion limit to prevent RecursionError
sys.setrecursionlimit(5000000)

threads = 4

with Pool(threads) as pool:
    for result in pool.map(process_region_link, region_links):
        result_table.extend(result)

# Change recursion limit to default value
sys.setrecursionlimit(recursion_limit)

In [13]:
os.makedirs('export', exist_ok = True)

In [14]:
results_df = pd.DataFrame(data = result_table, columns = data_table_headers)

In [15]:
results_df.to_csv('export/elections.csv', header = False, index = False)

In [16]:
df = results_df.set_index(additional_headers)

In [17]:
df

Число избирателей, включенных в список избирателей   \
Регион                     ТИК        УИК                                                         
Республика Адыгея (Адыгея) Адыгейская 1                                                  2256     
                                      2                                                  2700     
                                      3                                                  2858     
                                      4                                                  2066     
                                      5                                                   700     
...                                                                                       ...     
Территория за пределами РФ None       8433                                                436     
                                      8434                                               1141     
                                      8436                                                 78     
                                      8437                                                848     
                                      8443                                               2004     

                                            Число избирательных бюллетеней, полученных участковой избирательной комиссией  \
Регион                     ТИК        УИК                                                                                   
Республика Адыгея (Адыгея) Адыгейская 1                                                  2181                               
                                      2                                                  2633                               
                                      3                                                  2752                               
                                      4                                                  2034                               
                                      5                                                   714                               
...                                                                                       ...                               
Территория за пределами РФ None       8433                                               2500                               
                                      8434                                               3000                               
                                      8436                                               1507                               
                                      8437                                               2500                               
                                      8443                                               4000                               

                                            Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно  \
Регион                     ТИК        УИК                                                                                    
Республика Адыгея (Адыгея) Адыгейская 1                                                     0                                
                                      2                                                     0                                
                                      3                                                     0                                
                                      4                                                     0                                
                                      5                                                     0                                
...                                                                                       ...                                
Территория за пределами РФ None       8433                                                  0                             

In [18]:
df.to_csv('export/elections_index.csv')